# 케라스와 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기

- TensorFlow Hub 및 Keras를 사용한 전이 학습의 기본적인 응용
- "영화 리뷰를 사용한 텍스트 분류"와 동일한 IMDB 데이터이나 keras.dataset가 아닌 tensorflow_datasets 사용
- TFHub로부터 훈련된 모델을 로드하기 위한 라이브러리인 tensorflow_hub를 사용

### 중요 모듈 import

In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


### IMDB 데이터셋 다운로드

- 전체 학습 데이터의 40%를 검증용 데이터 세트로 사용
- tfds.load는 tf.data.Dataset 타입을 반환

In [2]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Metal device set to: Apple M1


2022-02-04 10:01:29.439089: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-04 10:01:29.439310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

### 데이터 탐색

- 데이터 세트는 전처리된 정수 배열
- 정수는 영화 리뷰에 나타나는 단어
- 레이블은 0(부정적) 또는 1(긍정적)
- tf.data.Dataset.batch 함수는 전체 데이터 세트를 파라미터로 전달한 batch_size 크기만큼 나뉘어진 Dataset로 만들어준다.
- python의 iter 함수는 iterator를 반환하고 next 함수는 호출할 때마다 iterator를 순회하면서 실행한다.

> 아래 코드는 train_data를 10개씩 나누어 저장시킨 후 iterator로 반들어 첫 번째(next를 한 번만 호출했으므로) 10개를 불러오는 코드다.

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

2022-02-04 10:01:29.515366: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-04 10:01:29.539297: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

### 모델 구성

- 고려사항
    - 텍스트를 어떻게 표현할 것인가
    - 모델에서 얼마나 많은 층을 사용할 것인가
    - 각 층에서는 얼마나 많은 은닉 유닛을 사용할 것인가
    
- 텍스트를 표현하는 방법 중 한가지는 임베딩 벡터로 변환하는 것
- 임베딩 레이어는 첫 번째 레이어로만 사용 가능
- 임베딩의 장점
    - 텍스트 전처리에 대해 걱정할 필요가 없다.
    - 전이 학습에 따른 이점이 있다.
    - 임베딩 벡터는 고정 크기이기 때문에 처리 과정이 단순해진다.
- 이 예에서는 google/nnlm-en-dim50/2라고 하는 TensorFlow Hub에서 사전 훈련된 텍스트 임베딩 모델을 사용

In [6]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

2022-02-04 10:09:21.383914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 10:09:21.564423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

- 전체 모델 생성

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


- TensorFlow Hub 레이어: 이 레이어는 사전 훈련된 저장된 모델을 사용하여 문장을 임베딩 벡터에 매핑합니다. 사용 중인 사전 훈련된 텍스트 임베딩 모델(google/nnlm-en-dim50/2)은 **문장을 토큰으로 분할**하고 각 토큰을 임베딩한 다음 임베딩을 결합합니다. 결과적인 차원은 (num_examples, embedding_dimension)입니다. 이 NNLM 모델의 경우에는 embedding_dimension은 50입니다.
- Dense 레이어: 16개의 은닉 유닛(hidden unit)을 가진 완전 연결 층(Dense)
- Dense 레이어: 최종 출력층, 하나의 출력 노드를 가진 완전 연결 층, sigmoid 활성화 함수를 사용하므로 확률 또는 신뢰도 수준을 표현하는 0~1 사이의 실수가 출력됩.

### 손실 함수와 옵티마이저

- 모델이 로짓을 출력하므로 binary_crossentropy를 손실함수로 사용 (확률을 다루는데 적합)

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### 모델 훈련

- 512개 샘플의 미니 배치에서 10개 epoch 동안 모델을 훈련

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10


2022-02-04 10:21:20.764825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


30/30 [==============================] - ETA: 0s - loss: 0.6803 - accuracy: 0.5118

2022-02-04 10:21:27.136092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


30/30 [==============================] - 7s 188ms/step - loss: 0.6803 - accuracy: 0.5118 - val_loss: 0.6540 - val_accuracy: 0.5409
Epoch 2/10
30/30 [==============================] - 5s 179ms/step - loss: 0.6085 - accuracy: 0.6159 - val_loss: 0.5665 - val_accuracy: 0.6712
Epoch 3/10
30/30 [==============================] - 5s 177ms/step - loss: 0.4878 - accuracy: 0.7595 - val_loss: 0.4546 - val_accuracy: 0.7806
Epoch 4/10
30/30 [==============================] - 5s 174ms/step - loss: 0.3515 - accuracy: 0.8579 - val_loss: 0.3646 - val_accuracy: 0.8282
Epoch 5/10
30/30 [==============================] - 5s 176ms/step - loss: 0.2452 - accuracy: 0.9085 - val_loss: 0.3227 - val_accuracy: 0.8598
Epoch 6/10
30/30 [==============================] - 5s 174ms/step - loss: 0.1771 - accuracy: 0.9399 - val_loss: 0.3058 - val_accuracy: 0.8680
Epoch 7/10
30/30 [==============================] - 5s 174ms/step - loss: 0.1294 - accuracy: 0.9608 - val_loss: 0.3023 - val_accuracy: 0.8716
Epoch 8/10
30/30 

### 모델 평가

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.3563 - accuracy: 0.8543 - 1s/epoch - 24ms/step
loss: 0.356
accuracy: 0.854
